# Setup

In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.12.0


# MNIST Example

The below uses the MNIST database of handwritten digits:

http://yann.lecun.com/exdb/mnist/

## Load a dataset

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). 

The pixel values of the images range from 0 to 255. Scale these values to a range of 0 to 1 by dividing the values by `255.0`. This also converts the sample data from integers to floating-point numbers.

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Build a machine learning model

Build a [tf.keras.Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) model:

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

[Sequential](https://www.tensorflow.org/guide/keras/sequential_model) is useful for stacking layers where each layer has one input [tensor](https://www.tensorflow.org/guide/tensor) and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the [Flatten](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), [Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), and [Dropout](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers.

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.0907826 ,  0.02561883,  0.759544  , -0.01242453,  0.25221872,
         0.04480809,  0.16548052,  0.39588165,  0.21627727, -0.63472855]],
      dtype=float32)

The [tf.nn.softmax](https://www.tensorflow.org/api_docs/python/tf/nn/softmax) function converts these logits to probabilities for each class:

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.09113245, 0.08538327, 0.17787398, 0.08219601, 0.10709862,
        0.08703753, 0.09820054, 0.12364481, 0.10331769, 0.04411513]],
      dtype=float32)

Define a loss function for training using [losses.SparseCategoricalCrossentropy](https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy):

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.4414158

Before you start training, configure and compile the model using Keras [Model.compile](https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile). Set the [optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the [Model.fit](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) method to adjust your model parameters and minimize the loss:

In [12]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2990 - accuracy: 0.9126
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1437 - accuracy: 0.9572
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1063 - accuracy: 0.9680
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0854 - accuracy: 0.9728
Epoch 5/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0740 - accuracy: 0.9762


The [Model.evaluate](https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate) method checks the model's performance, usually on a [validation set](https://developers.google.com/machine-learning/glossary#validation-set) or [test set](https://developers.google.com/machine-learning/glossary#test-set).

In [13]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0691 - accuracy: 0.9788 - 358ms/epoch - 1ms/step


[0.06912174820899963, 0.9787999987602234]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [14]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [15]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.0434395e-06, 2.2819911e-07, 1.7241711e-05, 3.1198311e-04,
        9.0869022e-12, 8.2517471e-07, 4.4351111e-12, 9.9965262e-01,
        1.6983607e-06, 1.3372014e-05],
       [6.2869283e-09, 2.2103530e-03, 9.9778521e-01, 4.4468497e-06,
        4.4686422e-15, 4.1234513e-10, 4.0277706e-10, 5.1603195e-14,
        1.6946629e-08, 1.6961159e-14],
       [4.4979103e-08, 9.9904364e-01, 2.7563493e-04, 3.1229686e-06,
        8.9194167e-05, 4.8298311e-06, 1.7517121e-05, 4.9518654e-04,
        7.0532558e-05, 2.4356277e-07],
       [9.9992597e-01, 7.7108533e-09, 5.4130254e-05, 1.1515327e-07,
        7.7532682e-09, 2.1377681e-07, 2.2265076e-06, 1.7082326e-05,
        1.6351828e-08, 2.0325712e-07],
       [1.0946256e-06, 1.9718034e-10, 1.5284771e-06, 5.8868638e-10,
        9.9792743e-01, 4.8041797e-09, 8.1433456e-07, 2.5234202e-05,
        7.5788272e-08, 2.0438107e-03]], dtype=float32)>

## Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/sequential_model) API.

For more examples of using Keras, check out the [tutorials](https://www.tensorflow.org/tutorials/keras/). To learn more about building models with Keras, read the [guides](https://www.tensorflow.org/guide/keras). If you want learn more about loading and preparing data, see the tutorials on [image data loading](https://www.tensorflow.org/tutorials/load_data/images) or [CSV data loading](https://www.tensorflow.org/tutorials/load_data/csv).